In [5]:
import numpy as np
from scipy.optimize import minimize

# Define known parameters
rho1 = 1600; mu1 = 40e-3; N1 = 28000; D1 = 60e-3; M1 = 0.600; P1 = 26e5
rho2 = 1159.5; mu2 = 9.5592e-3
g = 9.81  # Acceleration due to gravity in m/s^2

# Initial guesses for unknowns
x0 = np.array([5500, 120e-3, 0.4, 5e5])

# Define lower and upper bounds
lb = [2000, 30e-3, 0.05, 1e5]  # Lower bounds
ub = [20000, 150e-3, 0.6, 4e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]

# Define the objective function (sum of squared residuals)
def objective_function(x):
    N2, D2, M2, P2 = x

    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = (M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = (g * P1/(rho1*g) / (N1**2 * D1**2)) - (g * P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Sum of squared residuals
    return F1**2 + F2**2 + F3**2 + F4**2

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
result = minimize(objective_function, x0, bounds=bounds, method='SLSQP', options={'disp': True, 'maxiter': 2000, 'ftol': 1e-16})

# Display the solution and residuals
N2, D2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
print(f"Model's Shaft Speed is {N2:.2f}")

print('Objective function value:')
print(result.fun)


Inequality constraints incompatible    (Exit mode 4)
            Current function value: 3006109584005236.5
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
Solution:
[5.e+03 6.e-02 3.e-01 2.e+01]
Objective function value:
3006109584005236.5
